# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [26]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [27]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,17.91,94,100,5.89,SH,1736468686
1,1,iqaluit,63.7506,-68.5145,-16.15,84,100,1.54,CA,1736468687
2,2,west island,-12.1568,96.8225,23.99,94,75,4.12,CC,1736468688
3,3,port-aux-francais,-49.3500,70.2167,4.31,93,100,23.76,TF,1736468690
4,4,greymouth,-42.4667,171.2000,18.09,77,81,2.93,NZ,1736468691


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x="Lng", 
    y="Lat", 
    geo=True, 
    tiles="OSM", 
    frame_width=800, 
    frame_height=600, 
    size="Humidity", 
    color="City", 
    alpha=0.5
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 30)  # Temperature between 21 and 30°C
]
filtered_weather_df = filtered_weather_df[
    (filtered_weather_df["Humidity"] > 30) & (filtered_weather_df["Humidity"] < 80)  # Humidity between 30% and 80%
]
filtered_weather_df = filtered_weather_df[filtered_weather_df["Wind Speed"] < 10]  # Wind Speed below 10 m/s
filtered_weather_df = filtered_weather_df[filtered_weather_df["Cloudiness"] == 0]  # Cloudiness = 0%

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

# Display sample data
filtered_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
65,65,lompoc,34.6391,-120.4579,22.79,51,0,0.91,US,1736468761
85,85,cartagena,10.3997,-75.5144,26.79,78,0,4.63,CO,1736468530
113,113,nova sintra,14.8667,-24.7167,23.67,74,0,7.04,CV,1736468820
209,209,vila do maio,15.1333,-23.2167,23.56,70,0,7.47,CV,1736468937
234,234,port lincoln,-34.7333,135.8667,21.61,58,0,4.07,AU,1736468968
253,253,callao,-12.0667,-77.1500,24.84,68,0,5.66,PE,1736468989
289,289,tionk essil,12.7856,-16.5217,22.69,59,0,2.63,SN,1736469031
334,334,cidade velha,14.9167,-23.6167,22.62,69,0,3.74,CV,1736469089
373,373,rio grande,-32.0350,-52.0986,21.48,75,0,6.89,BR,1736469019
432,432,hadibu,12.6500,54.0333,21.94,71,0,4.57,YE,1736469207


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = None

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
65,65,lompoc,34.6391,-120.4579,22.79,51,0,0.91,US,1736468761,None
85,85,cartagena,10.3997,-75.5144,26.79,78,0,4.63,CO,1736468530,None
113,113,nova sintra,14.8667,-24.7167,23.67,74,0,7.04,CV,1736468820,None
209,209,vila do maio,15.1333,-23.2167,23.56,70,0,7.47,CV,1736468937,None
234,234,port lincoln,-34.7333,135.8667,21.61,58,0,4.07,AU,1736468968,None
253,253,callao,-12.0667,-77.1500,24.84,68,0,5.66,PE,1736468989,None
289,289,tionk essil,12.7856,-16.5217,22.69,59,0,2.63,SN,1736469031,None
334,334,cidade velha,14.9167,-23.6167,22.62,69,0,3.74,CV,1736469089,None
373,373,rio grande,-32.0350,-52.0986,21.48,75,0,6.89,BR,1736469019,None
432,432,hadibu,12.6500,54.0333,21.94,71,0,4.57,YE,1736469207,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters for hotel search
search_radius = 10000  # Radius in meters for searching hotels
search_params = {
    "categories": "accommodation.hotel",  # Hotel category filter
    "apiKey": geoapify_key  # Geoapify API key
    "limit": 100  # Limit to 100 results
}

# Print a message indicating the start of the hotel search
print("Starting hotel search...")

# Iterate through the rows of hotel_df DataFrame
for idx, city_row in hotel_df.iterrows():
    # Get latitude and longitude from the current row
    latitude = city_row['Lat']
    longitude = city_row['Lng']

    # Update search parameters with current city's location
    search_params["filter"] = f"circle:{longitude},{latitude},{search_radius}"
    search_params["bias"] = f"proximity:{longitude},{latitude}"

    # Define base URL for the API request
    api_url = "https://api.geoapify.com/v2/places"

    # Make API request with the defined parameters
    response = requests.get(api_url, params=search_params)

    # Convert the response to JSON format
    response_data = response.json()

    # Try to extract hotel name from the API response and update DataFrame
    try:
        hotel_df.loc[idx, "Hotel Name"] = response_data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, assign "No hotel found"
        hotel_df.loc[idx, "Hotel Name"] = "No hotel found"

    # Log the result of the search
    print(f"{hotel_df.loc[idx, 'City']} - Nearest hotel: {hotel_df.loc[idx, 'Hotel Name']}")

# Display updated hotel DataFrame
hotel_df

Starting hotel search...
lompoc - Nearest hotel: No hotel found
cartagena - Nearest hotel: No hotel found
nova sintra - Nearest hotel: No hotel found
vila do maio - Nearest hotel: No hotel found
port lincoln - Nearest hotel: No hotel found
callao - Nearest hotel: No hotel found
tionk essil - Nearest hotel: No hotel found
cidade velha - Nearest hotel: No hotel found
rio grande - Nearest hotel: No hotel found
hadibu - Nearest hotel: No hotel found
luanda - Nearest hotel: No hotel found
pisco - Nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
65,65,lompoc,34.6391,-120.4579,22.79,51,0,0.91,US,1736468761,No hotel found
85,85,cartagena,10.3997,-75.5144,26.79,78,0,4.63,CO,1736468530,No hotel found
113,113,nova sintra,14.8667,-24.7167,23.67,74,0,7.04,CV,1736468820,No hotel found
209,209,vila do maio,15.1333,-23.2167,23.56,70,0,7.47,CV,1736468937,No hotel found
234,234,port lincoln,-34.7333,135.8667,21.61,58,0,4.07,AU,1736468968,No hotel found
253,253,callao,-12.0667,-77.1500,24.84,68,0,5.66,PE,1736468989,No hotel found
289,289,tionk essil,12.7856,-16.5217,22.69,59,0,2.63,SN,1736469031,No hotel found
334,334,cidade velha,14.9167,-23.6167,22.62,69,0,3.74,CV,1736469089,No hotel found
373,373,rio grande,-32.0350,-52.0986,21.48,75,0,6.89,BR,1736469019,No hotel found
432,432,hadibu,12.6500,54.0333,21.94,71,0,4.57,YE,1736469207,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    x="Lng", 
    y="Lat", 
    geo=True, 
    tiles="OSM", 
    frame_width=800, 
    frame_height=600, 
    size="Humidity", 
    color="Hotel Name", 
    alpha=0.5, 
    hover_cols=["Hotel Name", "Country"]
)

# Display the map plot
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,Country)